# Part 4 - Data Preprocessing

## Overview
This notebook uses the dataset we accumulated from the Data Cleaning stage and conducts preprocessing using techniques discussed in class. I primarily do preprocessing on our raw dataset and create reusable functions for later analysis. You will find that many of the later notebooks include the same functions to preprocess the data (although I did later create a module/class to do this). There are also some tests of topic modeling visualizations that I later choose to discard from my results.


## Table of Contents
1. [Open our relevant tweets from local `.jsonl` file](#1)
2. [Descriptive Statistics](#2)
    1. [By Users](#2a)
    2. [By Mode](#2b)
    3. [Creation of `DataTransformation` Module](#2c)
3. [Preprocessing the tweets](#3)
4. [Tests of Topic Modeling Visualizations](#4)
5. [Conclusions + Post-Data Exploration Thoughts](#5)

## Open our relevant tweets from local `.jsonl` file <a href name id="1">
    
Now that we have a file with our full set of tweets, we can access them directly.

In [6]:
import sys

sys.path.append(".")

import helper.DataRetrieval as dr

In [7]:
import json

relevantTweets = []
with open('/students/jw10/cs315/all-tweets/covid-mobility-tweet-all-relevant.jsonl') as f:
    for line in f:
        relevantTweets.append(json.loads(line))

print("We have collected:", len(relevantTweets), "relevant tweets and it is stored in", type(relevantTweets), "format.")

We have collected: 9123 relevant tweets and it is stored in <class 'list'> format.


## Descriptive Statistics <a href name id="2">

To obtain descriptive statistics, we aggregate our tweets based on (1) users and (2) transportation mode.

### By Users <a href name id="2a">

In [8]:
# relevantTweets[0]

In [9]:
def getUniqueUsers(tweets):
    '''
    Returns a dictionary with users/tweets as keys/value pairs
    '''
    userDict = {}

    for tweet in relevantTweets:
        tweetID = tweet['id']
        user = tweet['user']['id_str'] # user ID
        if user not in userDict:
            userDict[user] = [tweetID]
        else:
            userDict[user].append(tweetID)

    return userDict

In [10]:
uniqueUsers = getUniqueUsers(relevantTweets)

In [11]:
print(len(uniqueUsers.keys()))

8931


These results tell us that most Transportation+COVID-19 tweets captured were tweeted by unique users. This is not particularly helpful in terms of gathering more insights into the top user engagement, so we choose to focus on the modes of transportation.

In [12]:
maxTweetCount, maxTweeter = 0, 'none'
for user in uniqueUsers:
    tweetCount = len(uniqueUsers[user])
    if tweetCount > maxTweetCount:
        maxTweetCount, maxTweeter = tweetCount, user
        
print("Highest tweet count for single user:", maxTweetCount, "\nID of user with highest tweet count:", maxTweeter)

Highest tweet count for single user: 22 
ID of user with highest tweet count: 252563614


### By transportation mode <a href name id="2b">


In [17]:
import pandas as pd
import inflect

def getKeywordsByCategory():
    '''
    Gets keywords from the .csv file and adds the pluralized keywords. Returns a set of all keywords by category
    '''
    p = inflect.engine()
    keywords = pd.read_csv('./cs315_keywords.csv')
    # lowercase all keyword strings
    keywords['public transport'] = keywords['public transport'].str.lower()
    keywords['motorized'] = keywords['motorized'].str.lower()
    keywords['non-motorized'] = keywords['non-motorized'].str.lower()

    transit = [word for word in keywords['public transport'] if not pd.isna(word)]
    motorized = [word for word in keywords['motorized'] if not pd.isna(word)]
    nonmotorized = [word for word in keywords['non-motorized'] if not pd.isna(word)]
    
    transitPlural = [p.plural(word) for word in transit] # not exhaustive, but captures some meaning
    motorizedPlural = [p.plural(word) for word in motorized] # not exhaustive, but captures some meaning
    nonmotorizedPlural = [p.plural(word) for word in nonmotorized] # not exhaustive, but captures some meaning
    
    return (set(transit+transitPlural), set(motorized+motorizedPlural), set(nonmotorized+nonmotorizedPlural))


In [34]:
transit, motorized, nonmotorized = getKeywordsByCategory()

print(transit)
print(motorized)
print(nonmotorized)
# get the length of the keywords + plurals set
print(len(transit.union(motorized).union(nonmotorized)))

{'mass transits', 'railroad', 'shuttles', 'lines', 'amtrak', 'transportations', 'green line', 'heavy rail', 'caltrains', 'line', 'mtas', 'metro', 'caltrain', 'silver line', 'metrorail', 'bus', 'monorail', 'streetcar', 'bart', 'cable car', 'transports', 'blue lines', 'rapid transits', 'light rail', 'monorails', 'mta', 'metropolitans', 'railways', 'muni', 'blue line', 'metropolitan', 'red lines', 'rail', 'mbta', 'mbtas', 'barts', 'light rails', 'railway', 'heavy rails', 'railroads', 'streetcars', 'buse', 'commuter rails', 'subways', 'metrorails', 'rapid transit', 'silver lines', 'cable cars', 'rails', 'amtraks', 'mass transportations', 'metros', 'transit', 'munis', 'subway', 'green lines', 'buses', 'station', 'mass transportation', 'shuttle', 'stations', 'trolley', 'transport', 'mass transit', 'transportation', 'trolleys', 'commuter rail', 'transits', 'red line'}
{'honda', 'chevrolets', 'scooter', 'jeep', 'highways', 'toyotas', 'uber', 'lincolns', 'infiniti', 'routes', 'subaru', 'engine'

In [35]:
# copied from Data Cleaning notebook
def returnLoweredTweetText(tweet):
    '''
    Returns the non-truncated tweet text in lower case
    '''
    if "retweeted_status" not in tweet: 
        if 'full_text' not in tweet:
            return tweet['text'].lower()
        else:
            return tweet['full_text'].lower()
    else:
        if 'full_text' not in tweet['retweeted_status']:
            return tweet['retweeted_status']['text'].lower()
        else:
            return tweet['retweeted_status']['full_text'].lower()
        
def printTweetText(tweet):
    '''
    Prints out tweet text
    '''
    print('*** Tweet ***')
    if "retweeted_status" not in tweet: 
        if 'full_text' not in tweet:
            print(tweet['text'])
        else:
            print(tweet['full_text'])
    else:
        if 'full_text' not in tweet['retweeted_status']:
            print(tweet['retweeted_status']['text'])
        else:
            print(tweet['retweeted_status']['full_text'])



In [36]:
def extractMode(tweet, transit, motorized, nonmotorized):
    '''
    Returns the category of transit given the keywords that it is in. Tweets containing multiple categories are 
    added to each.
    '''
    modes = set()
    tweetText = returnLoweredTweetText(tweet)
    tweetTextSet = set(tweetText.split())
    if transit.intersection(tweetTextSet):
        modes.add('transit')
    if motorized.intersection(tweetTextSet):
        modes.add('motorized')
    if nonmotorized.intersection(tweetTextSet):
        modes.add('nonmotorized')

    return modes

def getTweetsByTransportationMode(tweets, categories):
    '''
    Returns a dictionary with transport-mode/tweets as keys/value pairs
    '''
    modeDict = {}
    transit, motorized, nonmotorized = categories

    for tweet in tweets:
#         tweetID = tweet['id']
        modes = extractMode(tweet, transit, motorized, nonmotorized)
        for mode in modes:
            if mode not in modeDict:
                modeDict[mode] = [tweet]
            else:
                modeDict[mode].append(tweet)

    return modeDict

In [37]:
tweetsByMode = getTweetsByTransportationMode(relevantTweets, getKeywordsByCategory())

print("Transit", len(tweetsByMode['transit']))
print("Motorized", len(tweetsByMode['motorized']))
print("Nonomotorized", len(tweetsByMode['nonmotorized']))

Transit 3000
Motorized 3687
Nonomotorized 2507


In [52]:
# we do this in the Text Analysis notebook
# import matplotlib.pyplot as plt

# fig = plt.figure()
# ax = fig.add_axes([0,0,1,1])
# modes = ['Transit', 'Motorized', 'Non-Motorized']
# counts = [len(tweetsByMode['transit']),len(tweetsByMode['motorized']), len(tweetsByMode['nonmotorized'])]
# ax.bar(modes,counts)

# # customizations

# plt.show()

# plt.savefig('tweetsPerCategory.png')

### Creation of `DataTransformation` Module <a href name id="2c">

I have taken the above functions and created a module called `DataTransformation`. The class `TransportationMode` splits our tweets into their respective categories. We will use this to prepare our files for LIWC analysis.

In [39]:
transitDF = pd.DataFrame.from_dict(tweetsByMode['transit'])

print(transitDF.keys())

Index(['created_at', 'id', 'id_str', 'full_text', 'truncated',
       'display_text_range', 'entities', 'source', 'in_reply_to_status_id',
       'in_reply_to_status_id_str', 'in_reply_to_user_id',
       'in_reply_to_user_id_str', 'in_reply_to_screen_name', 'user', 'geo',
       'coordinates', 'place', 'contributors', 'retweeted_status',
       'is_quote_status', 'quoted_status_id', 'quoted_status_id_str',
       'quoted_status_permalink', 'retweet_count', 'favorite_count',
       'favorited', 'retweeted', 'lang', 'possibly_sensitive', 'quoted_status',
       'extended_entities', 'withheld_in_countries'],
      dtype='object')


In [40]:
import random

random.seed(5) # I'm doing this so that the sequence of random numbers is replicable

for i in range(10):
    tweet = random.choice(tweetsByMode['motorized'])
    printTweetText(tweet)
    print(80*'-')   

*** Tweet ***
rt @wef @mikequindazzi @antgrasso @fisher85m
Remote working and online shopping could drive 14 million cars off US roads – permanently https://t.co/flGl4HiptR #covid19 #sdi20 https://t.co/u7OM3XRPZa
--------------------------------------------------------------------------------
*** Tweet ***
Info for Pak Exporters/importers:

All ports are operational in Italy.
No route for merchandise/trade cargo is blocked.
No Pakistani export/import consignment is halted in Italy due to Corona Virus issue.
Plz let us know if any of your consignment gets stuck anywhere in Italy. https://t.co/MXzO7IU3Yb
--------------------------------------------------------------------------------
*** Tweet ***
@sergiart5_ @fjruiddiid @XxRaptor_xX @qLxke_ The chances of serious illness are pretty much negligible. And when it comes to death, pretty much nonexistent. You’re more likely to die in a car crash then die from COVID when vaccinated, but people still drive cars
--------------------------------

## Preprocessing the tweets <a href name id='3'>

Tweets are preprocessed in a similar fashion to that described in [this paper](https://www.cs.toronto.edu/~jstolee/projects/topic.pdf ), which also follows similar conventions to citations [3],[4] in that paper.


In [41]:
import urllib

def getCovidKeywordsList():
    url ='https://raw.githubusercontent.com/echen102/COVID-19-TweetIDs/master/keywords.txt'
    file = urllib.request.urlopen(url)

    covidKeywords = []
    for line in file:
        decoded_line = line.decode("utf-8").split()
        covidKeywords.append(decoded_line[0].lower())

    return covidKeywords

print(getCovidKeywordsList())

['coronavirus', 'koronavirus', 'corona', 'cdc', 'wuhancoronavirus', 'wuhanlockdown', 'ncov', 'wuhan', 'n95', 'kungflu', 'epidemic', 'outbreak', 'sinophobia', 'china', 'covid-19', 'corona', 'covid', 'covid19', 'sars-cov-2', 'covidー19', 'covd', 'pandemic', 'coronapocalypse', 'canceleverything', 'coronials', 'socialdistancingnow', 'social', 'socialdistancing', 'panicbuy', 'panic', 'panicbuying', 'panic', '14dayquarantine', 'duringmy14dayquarantine', 'panic', 'panic', 'panicshop', 'inmyquarantinesurvivalkit', 'panic-buy', 'panic-shop', 'coronakindness', 'quarantinelife', 'chinese', 'chinesevirus', 'stayhomechallenge', 'stay', 'sflockdown', 'dontbeaspreader', 'lockdown', 'lock', 'shelteringinplace', 'sheltering', 'staysafestayhome', 'stay', 'trumppandemic', 'trump', 'flattenthecurve', 'flatten', 'china', 'chinavirus', 'quarentinelife', 'ppeshortage', 'saferathome', 'stayathome', 'stay', 'stay', 'stayhome', 'getmeppe', 'covidiot', 'epitwitter', 'pandemie', 'wear', 'wearamask', 'kung', 'covid

In [48]:
import string
import nltk
from nltk.tokenize import TweetTokenizer
import re

stopwordsList = nltk.corpus.stopwords.words('english')
punctuation = string.punctuation
covidKeywordsList = getCovidKeywordsList()

def removeLinks(tweetString):
    '''Takes a string and removes web links from it'''
    tweetString = re.sub(r'http\S+', '', tweetString) # remove http links
    tweetString = re.sub(r'bit.ly/\S+', '', tweetString) # rempve bitly links
    tweetString = tweetString.strip('[link]') # remove [links]
    return tweetString

def cleanTweets(someTweets):
    """Given a string that it's a tweet or many tweets joined together,
    clean it up to use for further analysis.
    """
    # Your code here
    # 1) lowercase tweet words
    loweredTweets = someTweets.lower()
    
    # 2) tokenize tweets
    tweet_tokenizer = TweetTokenizer()
    tokens = tweet_tokenizer.tokenize(loweredTweets)
    
    # 3) Remove stopwords
    cleanTweets = [w for w in tokens if w not in stopwordsList]

    # 4) Remove punctuation
    cleanTweets = [w for w in cleanTweets if w not in punctuation]

    # Remove @
    cleanTweets = [w for w in cleanTweets if w[0] != '@']

    # Remove numbers
    cleanTweets = [w for w in cleanTweets if not w.isnumeric()]

    # 5) Remove COVID keywords
    cleanTweets = [w for w in cleanTweets if not (w in covidKeywordsList or w[1:] in covidKeywordsList)]
    
    # random symbols
    cleanTweets = [w for w in cleanTweets if '’' not in w]
    
    return cleanTweets

# def oldCleanTweets(someTweets):
#     """Given a string that it's a tweet or many tweets joined together,
#     clean it up to use for further analysis.
#     """
#     # Your code here
#     # 1) lowercase tweet words
#     loweredTweets = someTweets.lower()
    
#     # 2) tokenize tweets
#     tweet_tokenizer = TweetTokenizer()
#     tokens = tweet_tokenizer.tokenize(loweredTweets)
    
#     # 3) Remove stopwords
#     cleanTweets = [w for w in tokens if w not in stopwordsList]
    
#     # 4) Remove punctuation
#     cleanTweets = [w for w in cleanTweets if w not in punctuation]
    
#     # 5) Remove COVID keywords
#     cleanTweets = [w for w in cleanTweets if not (w in covidKeywordsList or w[1:] in covidKeywordsList)]
    
#     cleanTweets = [w for w in cleanTweets if '’' not in w]
    
#     return cleanTweets

We aggregate data into pseudodocuments based on the mode of transport used.

In [49]:
transitWords = ''
motorizedWords = ''
nonmotorizedWords = ''

words = {}
for mode in tweetsByMode:
    for tweet in tweetsByMode[mode]:
        text = returnLoweredTweetText(tweet)
        if mode not in words:
            words[mode] = text
        else:
            words[mode] += ' ' + text
    
    
words.keys()

dict_keys(['nonmotorized', 'motorized', 'transit'])

In [50]:
transitDoc = removeLinks(words['transit'])
nonmotorizedDoc = removeLinks(words['nonmotorized'])
motorizedDoc = removeLinks(words['motorized'])

cleanTransit = cleanTweets(transitDoc)
cleanNonmotorized = cleanTweets(nonmotorizedDoc)
cleanMotorized = cleanTweets(motorizedDoc)

type(cleanTransit)

list

In [51]:
from collections import Counter

print(Counter(cleanTransit).most_common(10))
print(Counter(cleanNonmotorized).most_common(10))
print(Counter(cleanMotorized).most_common(10))

[('transport', 663), ('station', 531), ('public', 509), ('bus', 495), ('people', 436), ('railway', 395), ('transportation', 360), ('workers', 320), ('city', 274), ('metro', 254)]
[('walk', 893), ('walking', 878), ('one', 608), ('around', 444), ('night', 438), ('people', 428), ('go', 311), ('think', 310), ('would', 301), ('right', 280)]
[('car', 943), ('road', 697), ('people', 557), ('traffic', 419), ('cars', 341), ('get', 323), ('roads', 296), ('city', 245), ('...', 230), ('vehicles', 228)]


### Tests of Topic Modeling Visualizations <a href name id='4'>

After trying out various libraries/methods for topic visualization, I found that topics are very difficult to decipher. Thus, we go reiterate the data preparation <--> modeling step, this time focusing on sentiment analysis.

[Can ignore below]

In [55]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation

import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()

# Create the TF vector representation, this only counts the terms in each document

tf_vectorizer = CountVectorizer(strip_accents = 'unicode',
                                stop_words = 'english',
                                lowercase = True,
                                token_pattern = r'\b[a-zA-Z]{3,}\b',
                                max_df = 0.5, 
                                min_df = 10)

dtm_tf = tf_vectorizer.fit_transform(cleanTransit)

tfidf_vectorizer = TfidfVectorizer(**tf_vectorizer.get_params())

dtm_tfidf = tfidf_vectorizer.fit_transform(cleanTransit)
print(dtm_tfidf.shape)

# for TFIDF DTM
lda_tfidf = LatentDirichletAllocation(n_components=10, random_state=0)
lda_tfidf.fit(dtm_tfidf)
pyLDAvis.sklearn.prepare(lda_tfidf, dtm_tfidf, tfidf_vectorizer)

(43146, 734)


/students/jw10/miniconda3/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/students/jw10/miniconda3/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/students/jw10/miniconda3/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/students/jw10/miniconda3/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3      0.440422  0.036026       1        1  11.299243
7     -0.072852  0.408785       2        1  10.456131
6     -0.062123 -0.025314       3        1  10.445816
1     -0.001371 -0.178611       4        1  10.390021
8     -0.056678 -0.025185       5        1  10.187481
9     -0.048480 -0.022946       6        1   9.793095
4     -0.048479 -0.110110       7        1   9.756565
5     -0.056544 -0.031947       8        1   9.356750
0     -0.042323 -0.021882       9        1   9.268716
2     -0.051572 -0.028818      10        1   9.046182, topic_info=          Term        Freq       Total Category  logprob  loglift
669  transport  441.000000  441.000000  Default  30.0000  30.0000
602    station  360.000000  360.000000  Default  29.0000  29.0000
58         bus  341.000000  341.000000  Default  28.0000  28.0000
480     public  340.000000  340.000000  Default  27.0000  27.0000
443     people  278.000000  278.000000  Default  26.0000  26.0000
..         ...         ...         ...      ...      ...      ...
461   possible   20.434211   21.300744  Topic10  -4.5273   2.3613
150    despite   20.434211   21.300744  Topic10  -4.5273   2.3613
604       step   20.434211   21.300744  Topic10  -4.5273   2.3613
64   cancelled   19.364357   20.230890  Topic10  -4.5810   2.3591
496       real   21.504452   29.323924  Topic10  -4.4762   2.0927

[332 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
5         8  0.968201       adds
8         9  0.959108   affected
14        9  0.959831        air
15        5  0.985132    airport
18        5  0.985132    allowed
...     ...       ...        ...
729       9  0.983362       year
730      10  0.986239      years
731       2  0.968434  yesterday
732       6  0.985151       york
733       4  0.976737       zero

[304 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 8, 7, 2, 9, 10, 5, 6, 1, 3])

In [56]:
pyLDAvis.sklearn.prepare(lda_tfidf, dtm_tfidf, tfidf_vectorizer, mds='mmds')

/students/jw10/miniconda3/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/students/jw10/miniconda3/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/students/jw10/miniconda3/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/students/jw10/miniconda3/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3      0.373777  0.252338       1        1  11.299243
7      0.116130  0.423736       2        1  10.456131
6     -0.329503 -0.242583       3        1  10.445816
1     -0.243771  0.408418       4        1  10.390021
8      0.192605 -0.423848       5        1  10.187481
9     -0.166258  0.152029       6        1   9.793095
4     -0.111554 -0.405712       7        1   9.756565
5     -0.461531  0.017997       8        1   9.356750
0      0.178489 -0.084196       9        1   9.268716
2      0.451616 -0.098179      10        1   9.046182, topic_info=          Term        Freq       Total Category  logprob  loglift
669  transport  441.000000  441.000000  Default  30.0000  30.0000
602    station  360.000000  360.000000  Default  29.0000  29.0000
58         bus  341.000000  341.000000  Default  28.0000  28.0000
480     public  340.000000  340.000000  Default  27.0000  27.0000
443     people  278.000000  278.000000  Default  26.0000  26.0000
..         ...         ...         ...      ...      ...      ...
461   possible   20.434211   21.300744  Topic10  -4.5273   2.3613
150    despite   20.434211   21.300744  Topic10  -4.5273   2.3613
604       step   20.434211   21.300744  Topic10  -4.5273   2.3613
64   cancelled   19.364357   20.230890  Topic10  -4.5810   2.3591
496       real   21.504452   29.323924  Topic10  -4.4762   2.0927

[332 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
5         8  0.968201       adds
8         9  0.959108   affected
14        9  0.959831        air
15        5  0.985132    airport
18        5  0.985132    allowed
...     ...       ...        ...
729       9  0.983362       year
730      10  0.986239      years
731       2  0.968434  yesterday
732       6  0.985151       york
733       4  0.976737       zero

[304 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 8, 7, 2, 9, 10, 5, 6, 1, 3])

In [58]:
dtm_tf = tf_vectorizer.fit_transform(cleanNonmotorized)

tfidf_vectorizer = TfidfVectorizer(**tf_vectorizer.get_params())

dtm_tfidf = tfidf_vectorizer.fit_transform(cleanNonmotorized)
print(dtm_tfidf.shape)

# for TFIDF DTM
lda_tfidf = LatentDirichletAllocation(n_components=10, random_state=0)
lda_tfidf.fit(dtm_tfidf)
pyLDAvis.sklearn.prepare(lda_tfidf, dtm_tfidf, tfidf_vectorizer, mds='mmds')

(33024, 460)


/students/jw10/miniconda3/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/students/jw10/miniconda3/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/students/jw10/miniconda3/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/students/jw10/miniconda3/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.014145  0.073608       1        1  11.093353
8      0.047954 -0.412754       2        1  10.761886
5     -0.364297 -0.135262       3        1  10.559462
6     -0.229003  0.385471       4        1  10.271619
4      0.335398 -0.317605       5        1  10.222592
9      0.398662 -0.040932       6        1   9.875182
2      0.097890  0.438333       7        1   9.840942
3     -0.243380 -0.380356       8        1   9.361527
7     -0.428598  0.134310       9        1   9.109332
0      0.371230  0.255189      10        1   8.904105, topic_info=           Term        Freq       Total Category  logprob  loglift
429        walk  529.000000  529.000000  Default  30.0000  30.0000
430     walking  552.000000  552.000000  Default  29.0000  29.0000
260       night  301.000000  301.000000  Default  28.0000  28.0000
279      people  243.000000  243.000000  Default  27.0000  27.0000
315       right  206.000000  206.000000  Default  26.0000  26.0000
..          ...         ...         ...      ...      ...      ...
448     working   15.426692   16.294147  Topic10  -4.3954   2.3640
0    absolutely   14.332601   15.200055  Topic10  -4.4689   2.3599
10         amid   14.332601   15.200055  Topic10  -4.4689   2.3599
71        cover   14.332601   15.200055  Topic10  -4.4689   2.3599
293        post   18.780088   32.505933  Topic10  -4.1987   1.8700

[331 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
0        10  0.921049  absolutely
4         4  0.960321     alabama
5         1  0.911221     allowed
8         5  0.920075     america
10       10  0.921049        amid
...     ...       ...         ...
452       4  0.974171        year
453       6  0.974825       years
456       3  0.960538         yes
458       7  0.966514        york
459       9  0.969620        zero

[302 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 9, 6, 7, 5, 10, 3, 4, 8, 1])

In [59]:
dtm_tf = tf_vectorizer.fit_transform(cleanMotorized)

tfidf_vectorizer = TfidfVectorizer(**tf_vectorizer.get_params())

dtm_tfidf = tfidf_vectorizer.fit_transform(cleanMotorized)
print(dtm_tfidf.shape)

# for TFIDF DTM
lda_tfidf = LatentDirichletAllocation(n_components=10, random_state=0)
lda_tfidf.fit(dtm_tfidf)
pyLDAvis.sklearn.prepare(lda_tfidf, dtm_tfidf, tfidf_vectorizer, mds='mmds')


(51572, 863)


/students/jw10/miniconda3/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/students/jw10/miniconda3/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/students/jw10/miniconda3/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/students/jw10/miniconda3/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
9      0.014597  0.072944       1        1  10.521873
8      0.050075 -0.412349       2        1  10.504530
3     -0.360050 -0.134250       3        1  10.356612
7     -0.225498  0.379440       4        1  10.250274
1      0.331165 -0.312664       5        1   9.964959
5      0.392235 -0.040662       6        1   9.922391
6      0.096084  0.441215       7        1   9.857262
4     -0.243146 -0.381527       8        1   9.721297
2     -0.431567  0.134813       9        1   9.451456
0      0.376105  0.253040      10        1   9.449346, topic_info=        Term        Freq       Total Category  logprob  loglift
97       car  548.000000  548.000000  Default  30.0000  30.0000
652     road  455.000000  455.000000  Default  29.0000  29.0000
552   people  336.000000  336.000000  Default  28.0000  28.0000
785  traffic  272.000000  272.000000  Default  27.0000  27.0000
266     ford  270.000000  270.000000  Default  26.0000  26.0000
..       ...         ...         ...      ...      ...      ...
663     sama   19.477344   20.343498  Topic10  -4.7706   2.3157
45       bad   18.457588   19.323741  Topic10  -4.8244   2.3134
861    young   18.457588   19.323741  Topic10  -4.8244   2.3134
17       aku   17.437831   18.303985  Topic10  -4.8812   2.3107
393      law   17.437831   18.303985  Topic10  -4.8812   2.3107

[330 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
2        10  0.982089    accident
3         2  0.980158   accidents
11        7  0.980028      advice
15        9  0.981966         air
17       10  0.928759         aku
...     ...       ...         ...
855       5  0.983659  wuhanvirus
857       8  0.985128        year
858       1  0.976004       years
859       6  0.962759         yes
861      10  0.931497       young

[300 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[10, 9, 4, 8, 2, 6, 7, 5, 3, 1])

In [67]:
corpus = list(cleanTransit) + list(cleanNonmotorized) + list(cleanMotorized)

dtm_tf = tf_vectorizer.fit_transform(corpus)

tfidf_vectorizer = TfidfVectorizer(**tf_vectorizer.get_params())

dtm_tfidf = tfidf_vectorizer.fit_transform(corpus)
print(dtm_tfidf.shape)

# for TFIDF DTM
lda_tfidf = LatentDirichletAllocation(n_components=10, random_state=0)
lda_tfidf.fit(dtm_tfidf)
pyLDAvis.sklearn.prepare(lda_tfidf, dtm_tfidf, tfidf_vectorizer, mds='mmds')


(127742, 1994)


/students/jw10/miniconda3/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/students/jw10/miniconda3/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/students/jw10/miniconda3/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/students/jw10/miniconda3/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.014945  0.074062       1        1  10.485973
4      0.049136 -0.414227       2        1  10.413136
7     -0.363539 -0.134986       3        1  10.377498
6     -0.228831  0.385512       4        1  10.247957
8      0.337371 -0.319085       5        1  10.146385
2      0.399876 -0.041887       6        1  10.064215
5      0.097029  0.444947       7        1   9.747761
3     -0.248383 -0.389556       8        1   9.715727
9     -0.437904  0.137526       9        1   9.631265
0      0.380300  0.257694      10        1   9.170084, topic_info=          Term        Freq       Total Category  logprob  loglift
1251    people  940.000000  940.000000  Default  30.0000  30.0000
227        car  614.000000  614.000000  Default  29.0000  29.0000
1912   walking  618.000000  618.000000  Default  28.0000  28.0000
1910      walk  562.000000  562.000000  Default  27.0000  27.0000
1490      road  531.000000  531.000000  Default  26.0000  26.0000
...        ...         ...         ...      ...      ...      ...
999       lost   55.458627   56.329224  Topic10  -4.8177   2.3736
445    despite   54.394162   55.264759  Topic10  -4.8371   2.3733
1689  students   54.394162   55.264759  Topic10  -4.8371   2.3733
897        key   53.329697   54.200294  Topic10  -4.8568   2.3730
1907   waiting   51.200767   52.071364  Topic10  -4.8976   2.3724

[331 rows x 6 columns], token_table=      Topic      Freq            Term
term                                 
6         5  0.978809          access
40        1  0.993416             air
43        7  0.989388         airport
50        8  0.989607         allowed
60       10  0.985459            amid
...     ...       ...             ...
1978      9  0.990086  wuhanpneumonia
1983      6  0.995973            year
1984      2  0.993184           years
1989      9  0.990346            york
1992      2  0.984868            zero

[302 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 5, 8, 7, 9, 3, 6, 4, 10, 1])

### Conclusions + Post-Data Exploration Thoughts <a href name id='5'>
I tried out a `biterm topic modeling` library but obtained quite strange results. Based on the pyLDAvis visualization, it did not seem likely that my corpus would obtain enough meaningful information to form coherent topics. This exploration led me to turn my focus toward sentiment analysis rather than topic modeling.

### [INCOMPLETE/INCOHERENT] LDA and Word Embeddings Analysis

In [88]:
import bitermplus as btm
import numpy as np
import pandas as pd

# IMPORTING DATA
texts = cleanTransit

# PREPROCESSING
# Obtaining terms frequency in a sparse matrix and corpus vocabulary
X, vocabulary, vocab_dict = btm.get_words_freqs(texts)
tf = np.array(X.sum(axis=0)).ravel()
# Vectorizing documents
docs_vec = btm.get_vectorized_docs(texts, vocabulary)
docs_lens = list(map(len, docs_vec))
# Generating biterms
biterms = btm.get_biterms(docs_vec)

# INITIALIZING AND RUNNING MODEL
model = btm.BTM(
    X, vocabulary, seed=12321, T=8, M=20, alpha=50/8, beta=0.01)
model.fit(biterms, iterations=20)
p_zd = model.transform(docs_vec)

# METRICS
perplexity = btm.perplexity(model.matrix_topics_words_, p_zd, X, 8)
coherence = btm.coherence(model.matrix_topics_words_, X, M=20)
# or
perplexity = model.perplexity_
coherence = model.coherence_

# LABELS
model.labels_
# or
btm.get_docs_top_topic(texts, model.matrix_docs_topics_)

100%|████████████████████████████████████████| 44958/44958 [00:00<00:00, 134097.89it/s]


,documents,label
0,’,0
1,understand,2
2,kept,2
3,saying,2
4,public,2
...,...,...
44953,must,2
44954,demand,2
44955,johnson,2
44956,’,0
